In [1]:
from dotenv import load_dotenv,find_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI

from langchain.agents import (AgentExecutor, create_tool_calling_agent, 
            create_openai_tools_agent, create_openai_functions_agent,create_xml_agent,
            create_json_chat_agent,create_structured_chat_agent, create_react_agent,
            create_self_ask_with_search_agent)
from langchain_community.tools.tavily_search import TavilySearchResults,TavilyAnswer
from langchain_core.prompts import ChatPromptTemplate
from langchain import hub
from langchain_core.tools import tool,StructuredTool

In [3]:
load_dotenv(find_dotenv("../.env"))

True

In [4]:
llmGemini=ChatGoogleGenerativeAI(model="gemini-2.0-flash-001")
llmOpenAI=ChatOpenAI(model="gpt-4o-mini")

<p>The core idea of agents is to use a language model to chose a sequence of actions to take. In chains, a sequence of actions is hardcoded (in code). In agents , a language model is used as a reasoning engine to determine which actions to take and in which order</p>

In [5]:
singlePrompt="Who is Lawrence Bishnoi?"
multiPrompt="As of today, what will be Sachin Tendulkar's age to the power 12?. Use both the tools: `tavily_search_results` as well as `exponentiate`"

<h3>Tool Calling with Agent</h3>

In [6]:
tools=[TavilySearchResults(name="tavily_search_results",max_results=2,)]

In [7]:
def exponentiate(base:int,exponent:int) -> float:
    """
        Exponentiate one number with another and return the output
        base: The Base Number
        exponent: The times you would exponentiate with
    """
    return float(base)**float(exponent)

exponent=StructuredTool.from_function(
    func=exponentiate,
    name="exponentiate",
    description="Useful when you need to find the exponent of any given base number"
)

# For Multi Input Tools
multiInputTools=[TavilySearchResults(name="tavily_search_results",max_results=2),exponent]

In [8]:
exponent.name

'exponentiate'

In [9]:
exponent.invoke(input={'base':'52','exponent':'12'})

3.908770064862502e+20

In [10]:
prompt=ChatPromptTemplate.from_messages(
    messages=[
        ("system","You are a helpful assistant."),
        ("placeholder","{chat_history}"),
        ("human","{input}"),
        ("placeholder","{agent_scratchpad}")
    ]
)

In [11]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant.'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad', optional=True)]

In [10]:
agent=create_tool_calling_agent(llm=llmOpenAI,tools=tools,prompt=prompt)
agentExecutor=AgentExecutor(agent=agent,tools=tools,verbose=True)

multiInputAgent=create_tool_calling_agent(llm=llmOpenAI,tools=multiInputTools,prompt=prompt)
multiAgentExecutor=AgentExecutor(agent=multiInputAgent,tools=multiInputTools,verbose=True)

In [11]:
responseSingleTool=agentExecutor.invoke(input={"input":singlePrompt})
responseSingleTool



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results` with `{'query': 'Lawrence Bishnoi'}`


[{'title': 'Lawrence Bishnoi - Wikipedia', 'url': 'https://en.wikipedia.org/wiki/Lawrence_Bishnoi', 'content': 'Lawrence Bishnoi (born Balkaran Brar; 12 February 1993) is an Indian gangster who attained notoriety as the boss of the "Bishnoi gang", which is reportedly linked to over 700 shooters operating across the world. Bishnoi has been imprisoned since 2014 and faces multiple criminal charges, including those for extortion and murder, however he has denied all allegations.\n\n## Early life and education [...] Wikipedia\nThe Free Encyclopedia\n\n## Contents\n\n# Lawrence Bishnoi\n\nPage semi-protected\n\n| Lawrence Bishnoi | |\n| --- | --- |\n| Born | Balkaran Brar   (1993-02-12) 12 February 1993 (age 32)  Fazilka district, Punjab, India |\n| Alma mater | DAV College (B.A.)  Panjab University (LL.B) |\n| Years active | 2010–present |\n| Judicial status | Imprisoned |\n|  

{'input': 'Who is Lawrence Bishnoi?',
 'output': 'Lawrence Bishnoi, born Balkaran Brar on February 12, 1993, is an Indian gangster known for being the boss of the "Bishnoi gang," which has been linked to a large network of over 700 shooters globally. He has been imprisoned since 2014 and faces multiple criminal charges, including extortion and murder, although he denies all allegations against him.\n\nBishnoi was born in the Fazilka district of Punjab, India, and completed his early education before moving to Chandigarh for higher studies. He has gained notoriety in both India and internationally, and his activities have attracted significant media attention.\n\nFor more detailed information, you can see the articles from [Wikipedia](https://en.wikipedia.org/wiki/Lawrence_Bishnoi) and [BBC News](https://www.bbc.co.uk/news/articles/c4gqyxwl27no).'}

In [12]:
responseMultiTool=multiAgentExecutor.invoke(input={"input":multiPrompt})
responseMultiTool



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results` with `{'query': 'Sachin Tendulkar date of birth'}`


[{'title': 'Sachin Tendulkar - Wikipedia', 'url': 'https://en.wikipedia.org/wiki/Sachin_Tendulkar', 'content': 'Sachin Ramesh Tendulkar (/ˌsʌtʃɪn tɛnˈduːlkər/ ⓘ; Marathi: (/wiki/Help:IPA/Marathi "Help:IPA/Marathi"); born 24 April 1973) is an Indian former international cricketer who captained the Indian national team. He is widely regarded as one of the greatest cricketers of all time, and is the holder of several world records, including being the all-time highest run "Run (cricket)")-scorer in both ODI and Test cricket, receiving the most player of the match awards in international cricket, and being [...] |  |  |\n| --- | --- |\n| Tendulkar in 2017 | |\n| Personal information | |\n| Full name | Sachin Ramesh Tendulkar |\n| Born | (1973-04-24) 24 April 1973 (age 52) Bombay, India |\n| Nickname |  Little Master  Master Blaster  |\n| Height | 165 cm (5 ft 5 in

{'input': "As of today, what will be Sachin Tendulkar's age to the power 12?. Use both the tools: `tavily_search_results` as well as `exponentiate`",
 'output': 'Sachin Tendulkar was born on April 24, 1973, which makes him 52 years old as of today. His age raised to the power of 12 is approximately \\( 3.91 \\times 10^{20} \\).'}

<h3> OpenAI Tools with Agents </h3>

In [13]:
prompt=hub.pull("hwchase17/openai-tools-agent")

In [210]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [211]:
agent=create_openai_tools_agent(llm=llmOpenAI,tools=tools,prompt=prompt)
agentExecutor=AgentExecutor(agent=agent,tools=tools,verbose=True)

multiInputAgent=create_openai_tools_agent(llm=llmOpenAI,tools=multiInputTools,prompt=prompt)
multiAgentExecutor=AgentExecutor(agent=multiInputAgent,tools=multiInputTools,verbose=True)

In [212]:
responseSingleTool=agentExecutor.invoke(input={"input":singlePrompt})
responseSingleTool



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results` with `{'query': 'Lawrence Bishnoi'}`


[{'url': 'https://www.reuters.com/world/who-is-lawrence-bishnoi-centre-row-between-india-canada-2024-10-15/', 'content': 'WHO IS LAWRENCE BISHNOI? The NIA has accused the 31-year-old law graduate, in jail since 2015, of running a trans-national crime syndicate. Born in the northern state of Punjab, Bishnoi is short'}, {'url': 'https://www.bbc.com/news/articles/c4gqyxwl27no', 'content': 'The Canadian police were referring to Lawrence Bishnoi, a 31-year-old gangster from India, now back in the spotlight domestically and internationally. Indian police say his gang is allegedly linked to the killing of a prominent politician in Mumbai at the weekend - gunmen shot dead 66-year-old Baba Siddique near his son’s office. Once among India’s most wanted, Bishnoi has been in prison since 2015, now held far from his native Punjab state in Gujarat. Indian police link Bishnoi\'s gang to B

{'input': 'Who is Lawrence Bishnoi?',
 'output': "Lawrence Bishnoi is a 31-year-old gangster from India. He is accused of running a trans-national crime syndicate and has been in jail since 2015. Bishnoi, originally from Punjab, has been in the spotlight domestically and internationally for his alleged involvement in criminal activities. Indian police link Bishnoi's gang to various criminal activities, including the killing of a prominent politician in Mumbai. He has attracted followers due to his money, style, and influence. If you need more information, feel free to ask."}

In [213]:
responseMultiTool=multiAgentExecutor.invoke(input={"input":multiPrompt})
responseMultiTool



> Entering new AgentExecutor chain...

Invoking: `exponentiate` with `{'base': 52, 'exponent': 12}`


3.908770064862502e+20
Invoking: `tavily_search_results` with `{'query': 'Sachin Tendulkar age in 2024'}`


[{'url': 'https://celebrityborns.com/biography/sachin-tendulkar/2586', 'content': '14. minutes. 05. seconds. 51 years, 5months, 26 days old age Sachin Tendulkar will turn 52 on 24 April, 2025. Only 6 months, 3 days, 7 hours,15 minutes has left for his next birthday. Sachin Tendulkar Born On.'}, {'url': 'https://wikibio.in/sachin-tendulkar/', 'content': 'As I stood there watching the Indian greats on the field, I kept telling myself that I needed to be a part of the action in the middle.”\n14-year-old Sachin Tendulkar as a ball boy during the 1987 Cricket World Cup\nSachin Tendulkar with Vinod Kambli in his school days\nSachin Tendulkar praying to Lord Ganesha\nSachin Tendulkar’s restaurant\nSachin Tendulkar with Bharat Ratna\nSachin Tendulkar with Michael Schumacher\nSachin Tend

{'input': "In the Year 2024, what will be Sachin Tendulkar's age to the power 12?. Use both the tools: `tavily_search_results` as well as `exponentiate`",
 'output': "Sachin Tendulkar will turn 52 on April 24, 2025. \n\nIn the year 2024, Sachin Tendulkar's age to the power of 12 will be approximately 3.91 x 10^20."}

<h3> OpenAI Functions with Agents (Use OpenAI Tools Agent in lieu of this) </h3>

In [214]:
prompt=hub.pull("hwchase17/openai-functions-agent")

C:\Users\MSI 1\.conda\envs\oaivenv\Lib\site-packages\langsmith\client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [215]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [17]:
agent=create_openai_functions_agent(llm=llmOpenAI,tools=tools,prompt=prompt)
agentExecutor=AgentExecutor(agent=agent,tools=tools,verbose=True)

multiInputAgent=create_openai_functions_agent(llm=llmOpenAI,tools=multiInputTools,prompt=prompt)
multiAgentExecutor=AgentExecutor(agent=multiInputAgent,tools=multiInputTools,verbose=True)

In [18]:
responseSingleTool=agentExecutor.invoke(input={"input":singlePrompt})
responseSingleTool



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results` with `{'query': 'Lawrence Bishnoi'}`


[{'title': 'Lawrence Bishnoi - Wikipedia', 'url': 'https://en.wikipedia.org/wiki/Lawrence_Bishnoi', 'content': 'Lawrence Bishnoi (born Balkaran Brar; 12 February 1993) is an Indian gangster who attained notoriety as the boss of the "Bishnoi gang", which is reportedly linked to over 700 shooters operating across the world. Bishnoi has been imprisoned since 2014 and faces multiple criminal charges, including those for extortion and murder, however he has denied all allegations.\n\n## Early life and education [...] Wikipedia\nThe Free Encyclopedia\n\n## Contents\n\n# Lawrence Bishnoi\n\nPage semi-protected\n\n| Lawrence Bishnoi | |\n| --- | --- |\n| Born | Balkaran Brar   (1993-02-12) 12 February 1993 (age 32)  Fazilka district, Punjab, India |\n| Alma mater | DAV College (B.A.)  Panjab University (LL.B) |\n| Years active | 2010–present |\n| Judicial status | Imprisoned |\n|  

{'input': 'Who is Lawrence Bishnoi?',
 'output': 'Lawrence Bishnoi, born Balkaran Brar on February 12, 1993, is an Indian gangster known for being the leader of the "Bishnoi gang." This gang is reportedly responsible for operating over 700 shooters worldwide. Bishnoi has been imprisoned since 2014 and faces numerous charges, including murder and extortion, which he has denied.\n\nHe was born in the Fazilka district of Punjab, India, and comes from a family with a law enforcement background, as his father was a former constable in the Haryana Police. Bishnoi completed his early education in Punjab before moving to Chandigarh for further studies.\n\nFor more detailed information, you can read about him on [Wikipedia](https://en.wikipedia.org/wiki/Lawrence_Bishnoi) or on a news article from the [BBC](https://www.bbc.co.uk/news/articles/c4gqyxwl27no).'}

In [218]:
responseMultiTool=multiAgentExecutor.invoke(input={"input":multiPrompt})
responseMultiTool



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results` with `{'query': 'Sachin Tendulkar age in 2024'}`


[{'url': 'https://wikibio.in/sachin-tendulkar/', 'content': 'As I stood there watching the Indian greats on the field, I kept telling myself that I needed to be a part of the action in the middle.”\n14-year-old Sachin Tendulkar as a ball boy during the 1987 Cricket World Cup\nSachin Tendulkar with Vinod Kambli in his school days\nSachin Tendulkar praying to Lord Ganesha\nSachin Tendulkar’s restaurant\nSachin Tendulkar with Bharat Ratna\nSachin Tendulkar with Michael Schumacher\nSachin Tendulkar writing with his left hand\nSachin Tendulkar with his wax statue at Madame Tussauds\nSachin Tendulkar with his Cricket Kit\nSachin Tendulkar’s Instagram post\nSachin Tendulkar Spartan Sports Controversy\nLove and passion for the game always helps you find new ways to practice, and above all to enjoy what you do.#FlashbackFriday pic.twitter.com/7UHH13fe0Q\n— Sachin Tendulk

{'input': "In the Year 2024, what will be Sachin Tendulkar's age to the power 12?. Use both the tools: `tavily_search_results` as well as `exponentiate`",
 'output': "In the year 2024, Sachin Tendulkar will be 51 years old. If we calculate Sachin Tendulkar's age to the power of 12, the result is approximately 309,629,344,375,621,400."}

<h3>XML Agent</h3>

In [19]:
prompt=hub.pull(owner_repo_commit="hwchase17/xml-agent-convo")

In [20]:
print(prompt.messages)

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input', 'tools'], input_types={}, partial_variables={}, template="You are a helpful assistant. Help the user answer any questions.\n\nYou have access to the following tools:\n\n{tools}\n\nIn order to use a tool, you can use <tool></tool> and <tool_input></tool_input> tags. You will then get back a response in the form <observation></observation>\nFor example, if you have a tool called 'search' that could run a google search, in order to search for the weather in SF you would respond:\n\n<tool>search</tool><tool_input>weather in SF</tool_input>\n<observation>64 degrees</observation>\n\nWhen you are done, respond with a final answer between <final_answer></final_answer>. For example:\n\n<final_answer>The weather in SF is 64 degrees</final_answer>\n\nBegin!\n\nPrevious Conversation:\n{chat_history}\n\nQuestion: {input}\n{agent_scratchpad}"), additional_kwargs={})]


In [21]:
agent=create_xml_agent(llm=llmOpenAI,tools=tools,prompt=prompt)
agentExecutor=AgentExecutor(agent=agent,tools=tools,verbose=True)

multiInputAgent=create_xml_agent(llm=llmOpenAI,tools=multiInputTools,prompt=prompt)
multiAgentExecutor=AgentExecutor(agent=multiInputAgent,tools=multiInputTools,verbose=True)

In [22]:
responseSingleTool=agentExecutor.invoke(input={"input":singlePrompt})
responseSingleTool



> Entering new AgentExecutor chain...
<tool>tavily_search_results</tool><tool_input>Lawrence Bishnoi biography[{'title': 'Lawrence Bishnoi Biography - Bio Man', 'url': 'https://bioman.com.ng/lawrence-bishnoi-biography/', 'content': 'Lawrence Bishnoi Biography Lawrence Bishnoi, born on February 12, 1993, in Ferozpur, Punjab, India, is a well-known gangster. He grew up in Abohar and completed his early education there. In 2010, he moved to Chandigarh to attend DAV College. Later, he joined Panjab University, where he earned a law degree (LL.B). During his university years, Bishnoi became involved in student politics. He met', 'score': 0.9092728}, {'title': 'Lawrence Bishnoi - Wikipedia', 'url': 'https://en.wikipedia.org/wiki/Lawrence_Bishnoi', 'content': 'Lawrence Bishnoi (born Balkaran Brar; 12 February 1993) is an Indian gangster who attained notoriety as the boss of the "Bishnoi gang", which is reportedly linked to over 700 shooters operating across the world.\n\nBishnoi has been im

{'input': 'Who is Lawrence Bishnoi?',
 'output': 'Lawrence Bishnoi, born Balkaran Brar on February 12, 1993, in Fazilka district, Punjab, India, is an Indian gangster and the leader of the "Bishnoi gang". He gained notoriety for his criminal activities, which include extortion and murder, and has been linked to over 700 shooters globally. Bishnoi was involved in student politics during his law studies at Panjab University. He\'s been imprisoned since 2014 and has denied all allegations against him, including assassination threats to Bollywood actor Salman Khan and planning the murder of Punjabi singer Sidhu Moose Wala.'}

In [23]:
responseMultiTool=multiAgentExecutor.invoke(input={"input":multiPrompt})
responseMultiTool



> Entering new AgentExecutor chain...
<tool>tavily_search_results</tool><tool_input>Sachin Tendulkar's date of birth[{'title': 'Sachin Tendulkar - Wikipedia', 'url': 'https://en.wikipedia.org/wiki/Sachin_Tendulkar', 'content': 'Sachin Ramesh Tendulkar (/ˌsʌtʃɪn tɛnˈduːlkər/ ⓘ; Marathi: (/wiki/Help:IPA/Marathi "Help:IPA/Marathi"); born 24 April 1973) is an Indian former international cricketer who captained the Indian national team. He is widely regarded as one of the greatest cricketers of all time, and is the holder of several world records, including being the all-time highest run "Run (cricket)")-scorer in both ODI and Test cricket, receiving the most player of the match awards in international cricket, and being [...] |  |  |\n| --- | --- |\n| Tendulkar in 2017 | |\n| Personal information | |\n| Full name | Sachin Ramesh Tendulkar |\n| Born | (1973-04-24) 24 April 1973 (age 52) Bombay, India |\n| Nickname |  Little Master  Master Blaster  |\n| Height | 165 cm (5 ft 5 in) |\n| Bat

ValidationError: 2 validation errors for exponentiate
base
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='50, 12', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/int_parsing
exponent
  Field required [type=missing, input_value={'base': '50, 12'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing

<h3>JSON Chat Agent</h3>

In [24]:
prompt=hub.pull(owner_repo_commit="hwchase17/react-chat-json")     

In [25]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations a

In [26]:
agent=create_json_chat_agent(llm=llmOpenAI,tools=tools,prompt=prompt)
agentExecutor=AgentExecutor(agent=agent,tools=tools,verbose=True)

multiInputAgent=create_json_chat_agent(llm=llmOpenAI,tools=multiInputTools,prompt=prompt)
multiAgentExecutor=AgentExecutor(agent=multiInputAgent,tools=multiInputTools,verbose=True)

In [27]:
responseSingleTool=agentExecutor.invoke(input={"input":singlePrompt})
responseSingleTool



> Entering new AgentExecutor chain...
```json
{
    "action": "tavily_search_results",
    "action_input": "Lawrence Bishnoi biography"
}
```[{'title': 'Lawrence Bishnoi Biography - Bio Man', 'url': 'https://bioman.com.ng/lawrence-bishnoi-biography/', 'content': 'Lawrence Bishnoi Biography Lawrence Bishnoi, born on February 12, 1993, in Ferozpur, Punjab, India, is a well-known gangster. He grew up in Abohar and completed his early education there. In 2010, he moved to Chandigarh to attend DAV College. Later, he joined Panjab University, where he earned a law degree (LL.B). During his university years, Bishnoi became involved in student politics. He met', 'score': 0.9092728}, {'title': 'Lawrence Bishnoi - Wikipedia', 'url': 'https://en.wikipedia.org/wiki/Lawrence_Bishnoi', 'content': 'Lawrence Bishnoi (born Balkaran Brar; 12 February 1993) is an Indian gangster who attained notoriety as the boss of the "Bishnoi gang", which is reportedly linked to over 700 shooters operating across th

{'input': 'Who is Lawrence Bishnoi?',
 'output': "Lawrence Bishnoi, born on February 12, 1993, in Fazilka district, Punjab, India, is an Indian gangster known for being the leader of the 'Bishnoi gang,' which is reportedly linked to over 700 shooters. He has been imprisoned since 2014 and faces multiple criminal charges, including extortion and murder, to which he has denied the allegations. He graduated with a law degree from Panjab University and became involved in student politics during his university years."}

In [28]:
responseMultiTool=multiAgentExecutor.invoke(input={"input":multiPrompt})
responseMultiTool



> Entering new AgentExecutor chain...
```json
{
    "action": "tavily_search_results",
    "action_input": "Sachin Tendulkar's age as of October 2023"
}
```[{'title': 'Sachin Tendulkar Height, Age, Wife, Children, Family, Biography ...', 'url': 'https://starsunfolded.com/sachin-tendulkar/', 'content': 'Entertainment\n       Bollywood\n       Hollywood\n   Journalists\n   Miscellaneous\n       Business\n       Journalists\n       Famous Personalities\n   Politician\n   Sports\n\nSachin Tendulkar Height, Age, Wife, Children, Family, Biography, Records & More\n===============================================================================\n\nQuick Info→\n\nAge: 52 Years\n\nCaste: Rajapur Saraswat Brahmin\n\nHeight: 5\' 5"\n\nImage 2: Sachin Tendulkar Close Up [...] present the Laureus Best Sporting Moment to Sachin Tendulkar" width="150" height="150" class="alignnone size-full wp-image-276673"><br><br><strong>2023:</strong> Sachin Tendulkar and Brian Lara joined the likes of Sir Donald 

ValidationError: 2 validation errors for exponentiate
base
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='52, 12', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/int_parsing
exponent
  Field required [type=missing, input_value={'base': '52, 12'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing

<h3>Structured Chat (Capable of Taking MultiInput Tools)</h3>

In [29]:
prompt=hub.pull(owner_repo_commit="hwchase17/structured-chat-agent")     

In [30]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['tool_names', 'tools'], input_types={}, partial_variables={}, template='Respond to the human as helpfully and accurately as possible. You have access to the following tools:\n\n{tools}\n\nUse a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).\n\nValid "action" values: "Final Answer" or {tool_names}\n\nProvide only ONE action per $JSON_BLOB, as shown:\n\n```\n{{\n  "action": $TOOL_NAME,\n  "action_input": $INPUT\n}}\n```\n\nFollow this format:\n\nQuestion: input question to answer\nThought: consider previous and subsequent steps\nAction:\n```\n$JSON_BLOB\n```\nObservation: action result\n... (repeat Thought/Action/Observation N times)\nThought: I know what to respond\nAction:\n```\n{{\n  "action": "Final Answer",\n  "action_input": "Final response to human"\n}}\n\nBegin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. R

In [31]:
agent=create_structured_chat_agent(llm=llmOpenAI,tools=tools,prompt=prompt)
agentExecutor=AgentExecutor(agent=agent,tools=tools,verbose=True)

multiInputAgent=create_structured_chat_agent(llm=llmOpenAI,tools=multiInputTools,prompt=prompt)
multiAgentExecutor=AgentExecutor(agent=multiInputAgent,tools=multiInputTools,verbose=True)

In [32]:
responseSingleTool=agentExecutor.invoke(input={"input":singlePrompt})
responseSingleTool



> Entering new AgentExecutor chain...
Thought: I need to find current and accurate information about Lawrence Bishnoi.
Action:
```json
{
  "action": "tavily_search_results",
  "action_input": {"query": "Lawrence Bishnoi"}
}
```[{'title': 'Lawrence Bishnoi - Wikipedia', 'url': 'https://en.wikipedia.org/wiki/Lawrence_Bishnoi', 'content': 'Lawrence Bishnoi (born Balkaran Brar; 12 February 1993) is an Indian gangster who attained notoriety as the boss of the "Bishnoi gang", which is reportedly linked to over 700 shooters operating across the world. Bishnoi has been imprisoned since 2014 and faces multiple criminal charges, including those for extortion and murder, however he has denied all allegations.\n\n## Early life and education [...] Wikipedia\nThe Free Encyclopedia\n\n## Contents\n\n# Lawrence Bishnoi\n\nPage semi-protected\n\n| Lawrence Bishnoi | |\n| --- | --- |\n| Born | Balkaran Brar   (1993-02-12) 12 February 1993 (age 32)  Fazilka district, Punjab, India |\n| Alma mater | DAV

{'input': 'Who is Lawrence Bishnoi?',
 'output': "Lawrence Bishnoi, born Balkaran Brar on February 12, 1993, is an Indian gangster known for being the boss of the 'Bishnoi gang.' He is linked to over 700 shooters globally and has faced multiple criminal charges, including extortion and murder, although he denies all allegations. Bishnoi has been imprisoned since 2014 and is currently considered a significant figure in organized crime in India."}

In [33]:
responseMultiTool=multiAgentExecutor.invoke(input={"input":multiPrompt})
responseMultiTool



> Entering new AgentExecutor chain...
Thought: To find Sachin Tendulkar's age as of today, I need to look up his birth date and calculate his current age. Then I'll use the exponentiation tool to raise his age to the power of 12.

Action:
```
{
  "action": "tavily_search_results",
  "action_input": {
    "description": "Sachin Tendulkar birthdate and current age",
    "title": "Query",
    "type": "string"
  }
}
```

ValidationError: 1 validation error for TavilyInput
query
  Field required [type=missing, input_value={'description': 'Sachin T...uery', 'type': 'string'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing

<h3>React Agent</h3>

In [34]:
prompt=hub.pull(owner_repo_commit="hwchase17/react")

In [35]:
prompt.get_prompts()

[PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')]

In [36]:
agent=create_react_agent(llm=llmOpenAI,tools=tools,prompt=prompt)
agentExecutor=AgentExecutor(agent=agent,tools=tools,verbose=True)

multiInputAgent=create_react_agent(llm=llmOpenAI,tools=multiInputTools,prompt=prompt)
multiAgentExecutor=AgentExecutor(agent=multiInputAgent,tools=multiInputTools,verbose=True)

In [37]:
responseSingleTool=agentExecutor.invoke(input={"input":singlePrompt})
responseSingleTool



> Entering new AgentExecutor chain...
Lawrence Bishnoi is a person of interest in the news, likely related to criminal activities or significant events. I should search for current information to provide a detailed answer.  
Action: tavily_search_results  
Action Input: "Lawrence Bishnoi 2023 news"  [{'title': 'Lawrence Bishnoi - Wikipedia', 'url': 'https://en.wikipedia.org/wiki/Lawrence_Bishnoi', 'content': "On 21 September 2023, Bishnoi claimed responsibility for the killing of Sukhdool Singh Gill (also known as Sukha Duneke), a Khalistani separatist.. [...] ### Salman Khan Threat\n\nIn 2018, Bishnoi's associate Sampath Nehra attempted an attack on Salman Khan, linked to the 1998 blackbuck poaching case, as the Bishnoi gang considers the Blackbuck sacred. Bishnoi later issued direct threats against Khan, stating that Khan would be killed in Jodhpur.\n\nIn November 2023, Bishnoi claimed responsibility for a shooting at actor-singer Gippy Grewal's home due to his alleged association 

{'input': 'Who is Lawrence Bishnoi?',
 'output': 'Lawrence Bishnoi is a prominent figure in organized crime, recently known for claiming responsibility for high-profile killings and threats, and is currently incarcerated in Sabarmati Jail, facing multiple serious charges.'}

In [38]:
responseMultiTool=multiAgentExecutor.invoke(input={"input":multiPrompt})
responseMultiTool



> Entering new AgentExecutor chain...
I need to find out Sachin Tendulkar's current age to calculate his age to the power of 12. Since today is October 2, 2023, I will search for his birth date to compute his age accurately.

Action: tavily_search_results  
Action Input: "Sachin Tendulkar birth date"  [{'title': 'Sachin Tendulkar - Wikipedia', 'url': 'https://en.wikipedia.org/wiki/Sachin_Tendulkar', 'content': 'Sachin Ramesh Tendulkar (/ˌsʌtʃɪn tɛnˈduːlkər/ ⓘ; Marathi: (/wiki/Help:IPA/Marathi "Help:IPA/Marathi"); born 24 April 1973) is an Indian former international cricketer who captained the Indian national team. He is widely regarded as one of the greatest cricketers of all time, and is the holder of several world records, including being the all-time highest run "Run (cricket)")-scorer in both ODI and Test cricket, receiving the most player of the match awards in international cricket, and being [...] |  |  |\n| --- | --- |\n| Tendulkar in 2017 | |\n| Personal information | |\n| 

ValidationError: 2 validation errors for exponentiate
base
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='(50, 12)', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/int_parsing
exponent
  Field required [type=missing, input_value={'base': '(50, 12)'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing